Test Homework3

ชื่อ_รหัส 

1.ณัชณศา_6521

2.ชัญญาภัค_6567

Import Library

In [1]:
import roboticstoolbox as rtb
import numpy as np

from spatialmath import SE3
from math import pi
from HW3_utils import *
from FRA333_HW3_6521_6567 import *

ข้อที่ 1 : Jacobian Matrix

Parameter of Robot

In [2]:
d1 = 0.0892
a2 = -0.425
a3 = -0.39243
d4 = 0.109
d5 = 0.093
d6 = 0.082

DH Parameter (Modify)

In [3]:
T3_e = SE3(a3-d6,-d5,d4) * SE3.RPY(0,-pi/2,0)

robot = rtb.DHRobot(
    [
        rtb.RevoluteMDH(d= d1 ,offset= pi), 
        rtb.RevoluteMDH(alpha= pi/2), 
        rtb.RevoluteMDH(a= a2),
    ]
    ,tool = T3_e, name="3R robot")

print(robot)

DHRobot: 3R robot, 3 joints (RRR), dynamics, modified DH parameters
┌────────┬───────┬────────────┬────────┐
│  aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├────────┼───────┼────────────┼────────┤
│    0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│    0.0 │ 90.0° │         q2 │    0.0 │
│ -0.425 │  0.0° │         q3 │    0.0 │
└────────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘



In [4]:
print(robot.jacob0)

<bound method DHRobot.jacob0 of DHRobot: 3R robot, 3 joints (RRR), dynamics, modified DH parameters
┌────────┬───────┬────────────┬────────┐
│  aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├────────┼───────┼────────────┼────────┤
│    0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│    0.0 │ 90.0° │         q2 │    0.0 │
│ -0.425 │  0.0° │         q3 │    0.0 │
└────────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘
>


Prove

In [5]:
def proofJacobian(q: list[float], robot: rtb.DHRobot) -> bool:
    # คำนวณ Jacobian จากฟังก์ชัน endEffectorJacobianHW3
    J_e = endEffectorJacobianHW3(q)  # Jacobian ที่ได้จากฟังก์ชันที่คุณเขียน

    # คำนวณ Jacobian จาก Robotics Toolbox (อ้างอิงเฟรมเดียวกัน: base frame)
    J_ertb = robot.jacob0(q)  # Jacobian อ้างอิงจาก base frame

    allow_error = 0.0001  # ค่าความคลาดเคลื่อนที่ยอมรับได้ในการเปรียบเทียบ
    print(q)
    print("-----------Jacobian จากโค้ด-----------")
    print(J_e)
    print("-----------Jacob0 จาก Robotics Toolbox------------")
    print(J_ertb)
    print("------------Jacobian ถูกหรือไม่------------")
    
    # เปรียบเทียบ Jacobian ทั้งสอง
    return np.allclose(J_e, J_ertb, atol=allow_error)

# สร้างค่า q แบบสุ่ม
q = np.random.rand(3) * 2 * np.pi
# เรียกใช้หุ่นยนต์ที่สร้างไว้แล้ว (เช่นหุ่นยนต์ 3 ข้อต่อ)
result = proofJacobian(q, robot)

print("Jacobian is correct:", result)

[1.31023103 6.05960819 4.80956437]
-----------Jacobian จากโค้ด-----------
[[-4.59867799e-01  1.48547530e-01  1.24271195e-01]
 [ 9.80525517e-03  5.57136274e-01  4.66086448e-01]
 [ 0.00000000e+00 -4.46870836e-01 -3.24488341e-02]
 [ 0.00000000e+00 -9.66244519e-01 -9.66244519e-01]
 [ 0.00000000e+00  2.57626802e-01  2.57626802e-01]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]
-----------Jacob0 จาก Robotics Toolbox------------
[[-4.59867799e-01  1.48547534e-01  1.24271199e-01]
 [ 9.80525517e-03  5.57136261e-01  4.66086438e-01]
 [ 0.00000000e+00 -4.46870826e-01 -3.24488320e-02]
 [-1.38777878e-17 -9.66244496e-01 -9.66244496e-01]
 [-4.16333634e-17  2.57626810e-01  2.57626810e-01]
 [ 1.00000000e+00  6.12323400e-17  6.12323400e-17]]
------------Jacobian ถูกหรือไม่------------
Jacobian is correct: True


Prove2

In [6]:
def proveSingularity(q: list[float], robot: rtb.DHRobot) -> bool:
    
    # ตรวจสอบ Singularities โดยใช้ฟังก์ชัน checkSingularityHW3
    is_singular_HW3 = checkSingularityHW3(q)  # ฟังก์ชันที่คุณให้มา
    # คำนวณ Jacobian จาก Robotics Toolbox
    J_ertb = robot.jacob0(q)  # Jacobian อ้างอิงจาก base frame
    
    # ส่วนที่เกี่ยวข้องกับการเคลื่อนที่เชิงเส้น (3x3 matrix)
    J_v_part = J_ertb[:3, :]  # เฉพาะส่วนการเคลื่อนที่เชิงเส้นของ Jacobian
    
    # คำนวณ Determinant ของ Jacobian
    determinant = np.linalg.det(J_v_part)
    
    # ตรวจสอบ Singularities จากค่า determinant
    is_singular_rtb = abs(determinant) < 1e-3

    # พิมพ์ค่าที่สุ่มและผลลัพธ์ที่เกี่ยวข้อง
    print("q ที่สุ่มได้:", q)
    print("ค่า Determinant จาก RTB:", determinant)
    print("อยู่ในจุด Singularities จาก RTB:", is_singular_rtb)
    print("อยู่ในจุด Singularities จาก HW3:", is_singular_HW3)

    # เปรียบเทียบผลลัพธ์จากทั้งสองฟังก์ชัน
    result = is_singular_HW3 == is_singular_rtb
    print("ผลลัพธ์ตรงกันหรือไม่:", result)
    
    return result

# สุ่มค่า q สำหรับหุ่นยนต์ 3 ข้อต่อ
q = np.random.rand(3) * 2 * np.pi
print("ผลลัพธ์สุดท้าย:", proveSingularity(q,robot))

ไม่อยู่ในจุด Singularity
q ที่สุ่มได้: [0.02850424 0.80425769 1.48722653]
ค่า Determinant จาก RTB: -0.018003396568623157
อยู่ในจุด Singularities จาก RTB: False
อยู่ในจุด Singularities จาก HW3: False
ผลลัพธ์ตรงกันหรือไม่: True
ผลลัพธ์สุดท้าย: True


Prove3

In [5]:
def proveComputeEffort(q: list[float], w: list[float]) -> bool:
    # เรียกใช้ฟังก์ชัน computeEffortHW3
    tau_result = computeEffortHW3(q, w)
    
    # พิสูจน์ว่าการคำนวณ torque ถูกต้องหรือไม่
    J_e = endEffectorJacobianHW3(q)
    J_t = np.transpose(J_e)

    # คำนวณ tau ด้วยวิธีการตรง
    tau_manual = J_t @ w

    # แปลงค่าเป็น numpy array สำหรับการเปรียบเทียบ
    tau_result = np.array(tau_result)
    tau_manual = np.array(tau_manual)

    # พิมพ์ผลลัพธ์เพื่อพิสูจน์
    print("Jacobian (J_e):\n", J_e)
    print("Transpose of Jacobian (J_t):\n", J_t)
    print("Force (w):", w)
    print("Torque จากฟังก์ชัน computeEffortHW3 (tau):", tau_result)
    print("Torque ที่คำนวณด้วยตนเอง (tau_manual):", tau_manual)

    # ตรวจสอบว่าผลลัพธ์จากฟังก์ชันตรงกับการคำนวณด้วยตนเองหรือไม่
    if np.allclose(tau_result, tau_manual):
        print("พิสูจน์สำเร็จ: ผลลัพธ์ถูกต้อง!")
        return True
    else:
        print("พิสูจน์ไม่สำเร็จ: ผลลัพธ์ไม่ถูกต้อง")
        return False

# เรียกใช้ฟังก์ชันพิสูจน์
q = np.random.rand(3) * 2 * np.pi
w = [10, 5, 2, 0, 0, 0]
proveComputeEffort(q, w)

Jacobian (J_e):
 [[-6.24570207e-02 -3.61132467e-01 -3.06639926e-01]
 [ 1.19383485e-01  1.79122288e-01  1.52093900e-01]
 [ 0.00000000e+00 -7.91978266e-02  3.41426733e-01]
 [ 0.00000000e+00  4.44345713e-01  4.44345713e-01]
 [ 0.00000000e+00  8.95855367e-01  8.95855367e-01]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]
Transpose of Jacobian (J_t):
 [[-6.24570207e-02  1.19383485e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [-3.61132467e-01  1.79122288e-01 -7.91978266e-02  4.44345713e-01
   8.95855367e-01  6.12323426e-17]
 [-3.06639926e-01  1.52093900e-01  3.41426733e-01  4.44345713e-01
   8.95855367e-01  6.12323426e-17]]
Force (w): [10, 5, 2, 0, 0, 0]
Torque จากฟังก์ชัน computeEffortHW3 (tau): [-0.02765278 -2.87410888 -1.6230763 ]
Torque ที่คำนวณด้วยตนเอง (tau_manual): [-0.02765278 -2.87410888 -1.6230763 ]
พิสูจน์สำเร็จ: ผลลัพธ์ถูกต้อง!


True